### Boilerplate

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from matplotlib import pyplot as plt
import json
from datetime import datetime, date, timedelta
import requests

import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *

from IPython.core.interactiveshell import InteractiveShell
from IPython.core.display import display, HTML
InteractiveShell.ast_node_interactivity = "all"
display(HTML('<style>.container {width:90% !important;}</style>'))

plt.style.use('fivethirtyeight')
%matplotlib inline

spark.sql('CREATE DATABASE IF NOT EXISTS football_games')
spark.sql('USE football_games')

os.chdir('../football_data')

AnalysisException: u'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

### Global Functions

In [5]:
def cast_dtypes(df):
    '''spark and numpy types don't play
    well together. cast as native python 
    types'''
    np_to_python_dtypes = {
        'int64': int,
        'float64': float,
        'object': str,
        'datetime64[ns]': str
    }

    for field, np_dtype in df.dtypes.to_dict().iteritems():
        df[field] = df[field].astype(np_to_python_dtypes[str(np_dtype)])
        
    return df

### Create __lines__ and __games__ DFs: load data and remove 2018 until it's complete

In [6]:
lines_df = pd.read_csv('all_lines.csv', index_col='Unnamed: 0')
games_df = pd.read_csv('game_metadata.csv')

In [7]:
lines_df = lines_df.rename(columns={'Season':'season'})
lines_df = lines_df[lines_df['season'] != 2018]
games_df = games_df[games_df['team0_link'].apply(lambda x: x.split('/')[-1].split('.')[0]) != '2018']

In [8]:
lines_df['season'].value_counts().sort_index()

2007    267
2008    267
2009    267
2010    267
2011    267
2012    267
2013    267
2014    267
2015    267
2016    267
2017    267
Name: season, dtype: int64

In [9]:
games_df['team0_link'].apply(lambda x: x.split('/')[-1].split('.')[0]).value_counts().sort_index()

2007    267
2008    267
2009    267
2010    267
2011    267
2012    267
2013    267
2014    267
2015    267
2016    267
2017    267
Name: team0_link, dtype: int64

In [10]:
lines_df.shape
games_df.shape

(2937, 26)

(2937, 23)

### Standardize Fields/Values Across Data Sources

#### Error in raw data (__lines__ DF)
* 3 games have home/visiting teams switched (note: these are 3 Superbowls so the home/away don't match the official home and visiting teams. the home/visiting team is always "team0" in football-ref. there is an assertion confirms this below).
  * https://www.pro-football-reference.com/boxscores/201602070den.htm
  * https://www.pro-football-reference.com/boxscores/201502010sea.htm
  * https://www.pro-football-reference.com/boxscores/201302030sfo.htm

In [11]:
switches = lines_df[lines_df['Date'].apply(lambda x: x in ['2016-02-07','2015-02-01','2013-02-03'])]

In [12]:
col_val_switch = ['ML_Fav','Open_Fav','Close_Fav','2H_Fav']

for c in col_val_switch:
    switches.loc[:, c] = switches.loc[:, c].apply(lambda x: {'V':'H', 'H':'V', 'pickem': 'pickem'}[x])

## H --> X
## V --> H
## X --> V
h_to_x = dict([(c, c.replace('H_','X_')) for c in switches.columns if c.startswith('H_')])
switches.rename(
    columns=h_to_x, inplace=True
)
v_to_h = dict([(c, c.replace('V_','H_')) for c in switches.columns if c.startswith('V_')])
switches.rename(
    columns=v_to_h, inplace=True
)
x_to_v = dict([(c, c.replace('X_','V_')) for c in switches.columns if c.startswith('X_')])
switches.rename(
    columns=x_to_v, inplace=True
)

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [13]:
idx = switches.index
lines_df = lines_df.drop(idx)
lines_df = lines_df.append(switches)

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [14]:
## Carolina, New England, and Baltimore should be home teams
lines_df[lines_df['Date'].apply(lambda x: x in ['2016-02-07','2015-02-01','2013-02-03'])].T

,1023,1290,1824
2H_Fav,H,H,V
2H_OU,22.5,pk,24.5
2H_Spread,4.5,24.5,7
Close_Fav,H,V,V
Close_OU,43.5,47.5,48
Close_Spread,5,0,4.5
Date,2016-02-07,2015-02-01,2013-02-03
H_Final,10,28,34
H_ML,-210,-110,170
H_Q1,0,0,7


#### Dates (__lines__ and __games__ DFs)

In [15]:
lines_df['Date'] = lines_df['Date'].apply(pd.to_datetime)

games_df['day_of_week'] = games_df['date'].apply(pd.to_datetime).dt.dayofweek
games_df['date'] = games_df['date'].apply(pd.to_datetime)

#### Team names (__lines__)

In [16]:
team_map = json.load(open('team_map_for_lines.json','r'))
for i, (k,v) in enumerate(team_map.iteritems()):
    print '{} --> {}'.format(k, v)
    if i == 5:
        break
        
for field in ['V_Team','H_Team']:
    lines_df[field] = lines_df[field].map(team_map)

Dallas --> Cowboys
NewYork --> Giants
NYGiants --> Giants
Minnesota --> Vikings
Denver --> Broncos
NewEngland --> Patriots


#### Create __games_lines__: prepare and then join __games__ and __lines__
* key is (date, team first alphabetically)

In [17]:
## cast dtypes to Spark compatible using global function cast_dtypes
games_df = cast_dtypes(games_df)
lines_df = cast_dtypes(lines_df)

In [18]:
def gen_game_key(x):
    '''generates a key for each game:
    (first alphabetical team name, date)'''
    team_a, team_b, dt = x
    return (sorted([team_a, team_b])[0], dt)

games_key_fields = ['team0_name','team1_name','date']
lines_key_fields = ['H_Team','V_Team', 'Date']

games_df['key'] = games_df[games_key_fields].apply(gen_game_key, axis=1)
lines_df['key'] = lines_df[lines_key_fields].apply(gen_game_key, axis=1)

Note: week 16 2018 lines not yet available

In [19]:
a = set(games_df['key'].apply(lambda x: '_'.join(x)).values.tolist())
b = set(lines_df['key'].apply(lambda x: '_'.join(x)).values.tolist())

len(a - b)
a - b

0

set()

In [20]:
len(b-a)
b-a

0

set()

In [21]:
games_lines_df = games_df.merge(
        lines_df, left_on='key', right_on='key'
    ).drop(
        lines_key_fields + ['key'], axis=1
    )

## ensure there are no join misses
lines_df.shape[0], games_df.shape[0], games_lines_df.shape[0]
assert lines_df.shape[0] == games_df.shape[0]
assert games_lines_df.shape[0] == games_df.shape[0]

(2937, 2937, 2937)

### Create __game_info__: load PFR game_info files
* contains metadata like turf and weather

In [22]:
## get the path for each game_info csv
game_info_csvs = map(
    lambda x: 'game_info/{}'.format(x), 
    os.listdir('game_info')
)
## read each game_info csv into pandas
game_info_dfs = map(
    lambda x: pd.read_csv(x, index_col='Unnamed: 0'), 
    game_info_csvs
)
## union all the DFs
game_info_df = reduce(
    lambda x,y: x.append(y).reset_index(drop=True), 
    game_info_dfs
)

In [23]:
for c in game_info_df.columns:
    game_info_df[c] = game_info_df[c].str.strip()

In [24]:
game_info_df.head()

,game_id,info,stat
0,201812090was,Won Toss,Giants (deferred)
1,201812090was,Roof,outdoors
2,201812090was,Surface,grass
3,201812090was,Weather,"33 degrees, wind 6 mph"
4,201812090was,Vegas Line,New York Giants -3.0


In [25]:
game_info_df['key_value'] = game_info_df[['info','stat']].apply(
    lambda (k,v): {k:v}, axis=1
)

game_info_dict = {}
for game in game_info_df['game_id'].unique():
    curr_game_df = game_info_df[game_info_df['game_id'] == game]
    game_info_dict[game] = {
            d.keys()[0]:d.values()[0] 
            for d in curr_game_df['key_value'].values
        }

In [26]:
game_info_colnames = {
    'Vegas Line': 'pfr_line',
    'Over/Under': 'pfr_ou',
    'Weather': 'prf_weather',
    'Roof': 'roof',
    'Surface': 'surface'
}

game_info = pd.DataFrame.from_dict(game_info_dict, orient='index')\
                .rename(columns=game_info_colnames)\
                .drop(['Won Toss','Won OT Toss','Super Bowl MVP'], axis=1)

In [27]:
game_info.head().T

,200709060clt,200709090buf,200709090cle,200709090dal,200709090gnb
surface,fieldturf,astroplay,grass,fieldturf,grass
pfr_line,Indianapolis Colts -5.5,Denver Broncos -3.0,Pittsburgh Steelers -4.5,Dallas Cowboys -6.5,Philadelphia Eagles -3.0
pfr_ou,53.5 (under),37.0 (under),36.5 (over),44.5 (over),42.5 (under)
roof,dome,outdoors,outdoors,outdoors,outdoors
prf_weather,NaN,"70 degrees relative humidity 93%, wind 5 mph, ...","71 degrees relative humidity 80%, wind 5 mph, ...","78 degrees relative humidity 85%, no wind, win...","62 degrees relative humidity 80%, wind 9 mph, ..."


### Create __games_lines_info__: join __games_lines__ with __game_info__ assert there are no join misses

In [28]:
## 2018 games
diffs = set(game_info.index.tolist()) - set(games_lines_df['game_id'].tolist())
set(map(lambda x: x[:4], diffs))

games_lines_info = games_lines_df.merge(game_info, left_on='game_id', right_index=True)

games_lines_df.shape[0], game_info.shape[0], games_lines_info.shape[0]

{'2018'}

(2937, 3177, 2937)

In [29]:
## TEMPORARILY TURNING OFF
# assert games_lines_df.shape[0] == game_info.shape[0]
# assert games_lines_df.shape[0] == games_lines_info.shape[0]

### Enrich __games_lines_info__

#### Infer the week number

In [30]:
## day of week -- 0==Monday, 6==Sunday
## game's date minus day of week --> week number
## since Monday games

def npdate_to_str(dt):
    '''convert np.datetime64 to 
    YYYY-MM-DD string'''
    return str(dt).split(' ')[0]

def make_week_id(x):
    '''given a date and day of week,
    return the "week id" which is the
    Sunday of that week. Monday (0) winds back
    a day. otherwise move toward Sunday (6).
    '''
    dt_str, dow = x
    dt = datetime.strptime(dt_str, '%Y-%m-%d')
    if dow == 6:
        week_id = dt
    elif dow == 0:
        week_id = dt - timedelta(days=1)
    else:
        delta = 6 - dow
        week_id = dt + timedelta(days=delta)
    
    return npdate_to_str(week_id)
    
games_lines_info['week_date'] = games_lines_info[['date','day_of_week']]\
                                .apply(make_week_id, axis=1)

In [31]:
games_lines_info['week_date'].head()

0    2007-09-09
1    2007-09-09
2    2007-09-09
3    2007-09-09
4    2007-09-09
Name: week_date, dtype: object

In [32]:
season_to_week = {}
for season in games_lines_info['season'].unique():
    season_to_week[season] = {}
    weeks = games_lines_info[games_lines_info['season'] == season]['week_date'].unique()
    for i, wk in enumerate(sorted(weeks)):
        season_to_week[season][wk] = i

In [33]:
games_lines_info['week_id'] = games_lines_info[['season','week_date']].apply(
        lambda x: season_to_week[x[0]][x[1]], axis=1
    )
games_lines_info.drop('week_date', axis=1, inplace=True)

In [34]:
games_lines_info['week_id'].head()

0    0
1    0
2    0
3    0
4    0
Name: week_id, dtype: int64

#### Timezones
* Make API calls to geonames.org to get timezone (GMT offset)

In [35]:
stadiums = pd.read_csv('stadium_lat_long.csv')

In [36]:
def get_tz(user, lat, lon):
    '''pandas UDF: given geonames username,
    lat, and lon, make a REST API call
    and return the timezone and GMT offset'''
    base_url = 'http://api.geonames.org/timezone'
    base_url += '?lat={LAT}&lng={LONG}&username={USER}'
    xml = requests.get(
            base_url.format(LAT=lat, LONG=lon, USER=user)
        ).text
    
    return BeautifulSoup(xml)\
                .findAll('gmtoffset')\
                [0].text

user = open('.username').read().strip()
stadiums['timezone'] = stadiums[['lat','long']].apply(
    lambda x: get_tz(user, *x), axis=1
)

In [37]:
stadiums['timezone'].value_counts()

-5.0    26
-6.0    18
-8.0    12
-7.0     3
0.0      2
Name: timezone, dtype: int64

In [38]:
# stadiums.to_csv('stadiums_w_tz.csv', index=False)

#### Stadium lat/long 
* for each stadium
* to be used for computing travel

In [39]:
## assert that stadiums lat/long csv has all stadiums in the games_df
assert not set(games_lines_info['stadium'].unique()) - set(stadiums['stadium'].unique())
before = games_lines_info.shape[0]
games_lines_info = games_lines_info.merge(
        stadiums, left_on='stadium', right_on='stadium'
    )

assert games_lines_info.shape[0] == before

In [40]:
games_lines_info[stadiums.columns].drop_duplicates().head(10)

,stadium,lat,long,timezone
0,RCA Dome,39.76361,-86.16333,-5.0
9,Ralph Wilson Stadium,42.77400,-78.78700,-5.0
74,Cleveland Browns Stadium,41.50611,-81.69944,-5.0
122,Texas Stadium,32.84000,-96.91100,-6.0
139,Lambeau Field,44.50139,-88.06222,-6.0
234,Reliant Stadium,29.68472,-95.41083,-6.0
292,Jacksonville Municipal Stadium,30.32389,-81.63750,-5.0
316,Hubert H. Humphrey Metrodome,44.97389,-93.25806,-6.0
342,Giants Stadium,40.81222,-74.07694,-5.0
391,McAfee Coliseum,37.75167,-122.20056,-8.0


#### Infer stadium --> home team (or neutral)
* TODO: clean this up
* how to get home team on neutral territory? (to compare to lines)
* jets/giants both home

In [41]:
## get a count of team games plabyed by stadium
tmp = games_lines_info.copy()
tmp['teams'] = tmp[['team0_link','team1_link']].apply(
    lambda x: map(lambda y: y.split('/')[2], x), 
    axis=1
)
stad = spark.createDataFrame(
        tmp[['teams','stadium','season']]
    ).select(
        F.explode('teams').alias('team'), 'stadium', 'season'
    ).groupby('stadium','season','team').count()\
     .groupby('stadium','season').pivot('team').agg(F.sum('count')).toPandas().fillna(0)

In [42]:
## compute the proportion of a stadium's games that each team played
## **assumption used: of a stadium has had more thna 6 games played
## and a team has >20% of games played there, that is the home team
## TODO: make this more elegant, maybe leveraging the Wiki package
key = ['stadium','season']
teams = list(set(stad.columns) - set(key))

stad['sum'] = stad[teams].sum(axis=1)

for t in teams:
    stad[t] /= stad['sum']
    
def find_home_team(props, teams):
    props, ngames = (props[:-1], props[-1])
    max_idx = [
        i for (i, t) in enumerate(props) 
        if (t > 0.2) & (ngames > 6)
    ]
    return map(lambda t: teams[t], max_idx)

stad['home_teams'] = stad[teams + ['sum']].apply(
    lambda x: find_home_team(x, teams), axis=1
)

In [43]:
# SPECIAL CASES (games relocated nearby)
## set Rogers Centre to buf
rog = stad[stad['stadium'] == 'Rogers Centre'].index
stad.loc[rog, 'home_teams'] = stad.loc[rog, 'season'].apply(lambda x: ['buf'])
stad.loc[rog, :]

## set TCF Bank Stadium to min
tcf = stad[stad['stadium'] == 'TCF Bank Stadium'].index
stad.loc[tcf, 'home_teams'] = stad.loc[tcf, 'season'].apply(lambda x: ['min'])
stad.loc[tcf, :]

,stadium,season,atl,buf,car,chi,cin,cle,clt,crd,...,rai,ram,rav,sdg,sea,sfo,tam,was,sum,home_teams
21,Rogers Centre,2011,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,2.0,[buf]
142,Rogers Centre,2012,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,2.0,[buf]
210,Rogers Centre,2009,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,[buf]
225,Rogers Centre,2013,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,[buf]
273,Rogers Centre,2010,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,[buf]
307,Rogers Centre,2008,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,[buf]


,stadium,season,atl,buf,car,chi,cin,cle,clt,crd,...,rai,ram,rav,sdg,sea,sfo,tam,was,sum,home_teams
33,TCF Bank Stadium,2010,0.0000,0.0,0.0000,0.500000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0000,2.0,[min]
48,TCF Bank Stadium,2015,0.0000,0.0,0.0000,0.055556,0.0,0.0,0.0,0.0,...,0.0,0.055556,0.0,0.055556,0.111111,0.0,0.0,0.0000,18.0,[min]
201,TCF Bank Stadium,2014,0.0625,0.0,0.0625,0.062500,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0625,16.0,[min]


In [44]:
stad[['stadium','season','home_teams']].sort_values(by=['stadium','season']).head()

,stadium,season,home_teams
153,AT&T Stadium,2013,[dal]
343,AT&T Stadium,2014,[dal]
360,AT&T Stadium,2015,[dal]
101,AT&T Stadium,2016,[dal]
222,AT&T Stadium,2017,[dal]


In [45]:
## take the previous DF, and tack on the 
## list of home teams
games_lines_info_homeaway = games_lines_info.merge(
    stad[['stadium','season','home_teams']],
    left_on=['stadium','season'],
    right_on=['stadium','season']
)
assert games_lines_info_homeaway.shape[0] == games_lines_info.shape[0]

In [46]:
games_lines_info_homeaway['team0'] = games_lines_info_homeaway['team0_link'].apply(lambda x: x.split('/')[2])
games_lines_info_homeaway['team1'] = games_lines_info_homeaway['team1_link'].apply(lambda x: x.split('/')[2])

games_lines_info_homeaway['team0_home_unof'] = games_lines_info_homeaway[['team0','home_teams']].apply(
    lambda x: x[0] in x[1],
    axis=1
)
games_lines_info_homeaway['team1_home_unof'] = games_lines_info_homeaway[['team1','home_teams']].apply(
    lambda x: x[0] in x[1],
    axis=1
)

In [47]:
def extract_home_team(x):
    team0, team1 = x
    if team0 == team1:
        return 'neutral'
    elif team0 is True:
        return 'team0'
    elif team1 is True:
        return 'team1'
    
games_lines_info_homeaway['home_team'] = games_lines_info_homeaway.loc[
        :, ['team0_home_unof','team1_home_unof']
    ].apply(
        extract_home_team, axis=1
    )
games_lines_info_homeaway.drop(
    ['team0_home_unof','team1_home_unof'], axis=1, inplace=True
)
games_lines_info_homeaway.rename(
    columns={'home_teams':'stadium_home_team'}, inplace=True
)

games_lines_info_homeaway['home_team'].value_counts()

team0      2898
neutral      39
Name: home_team, dtype: int64

In [48]:
## replace team0 with the team0 name
team0_idx = games_lines_info_homeaway[
                games_lines_info_homeaway['home_team'] == 'team0'
            ].index.tolist()

games_lines_info_homeaway.loc[team0_idx, 'home_team'] = games_lines_info_homeaway\
                                                            .loc[team0_idx, 'team0']

In [49]:
## still 39 neutral
games_lines_info_homeaway['home_team'].value_counts().tail()

mia        86
ram        86
rai        85
jax        84
neutral    39
Name: home_team, dtype: int64

In [50]:
assert games_lines_info_homeaway[
    (games_lines_info_homeaway['home_team'] == 'neutral')
    & (games_lines_info_homeaway[['H_Q1','H_Q2','H_Q3','H_Q4']].sum(axis=1) 
          != games_lines_info_homeaway['team0_score'])
    & (games_lines_info_homeaway[['V_Q1','V_Q2','V_Q3','V_Q4']].sum(axis=1) 
          != games_lines_info_homeaway['team1_score'])
].shape[0] == 0

In [51]:
games_lines_info_homeaway['H_score'] = games_lines_info_homeaway\
                                        [['H_Q1','H_Q2','H_Q3','H_Q4']].sum(axis=1)
games_lines_info_homeaway['V_score'] = games_lines_info_homeaway\
                                        [['V_Q1','V_Q2','V_Q3','V_Q4']].sum(axis=1)

games_lines_info_homeaway[
        games_lines_info_homeaway['home_team'] == 'neutral'
    ].loc[
        :, ['team0','team1','stadium','H_score',
            'team0_score','V_score','team1_score','stadium_home_team']
    ].sort_values(by='stadium').head(10)
                                 
games_lines_info_homeaway.drop(['H_score','V_score'], axis=1, inplace=True)

,team0,team1,stadium,H_score,team0_score,V_score,team1_score,stadium_home_team
2920,rai,nwe,Azteca Stadium,8,8,33,33,[]
2919,rai,htx,Azteca Stadium,27,27,20,20,[]
2142,gnb,pit,Cowboys Stadium,31,31,25,25,[dal]
1198,clt,nor,Dolphin Stadium,17,17,31,31,[mia]
1115,min,nyg,Ford Field,3,3,21,21,[det]
1146,buf,nyj,Ford Field,38,38,3,3,[det]
346,nyg,nyj,Giants Stadium,35,35,24,24,"[nyj, nyg]"
2797,car,den,Levi's Stadium,10,10,24,24,[sfo]
1994,nwe,nyg,Lucas Oil Stadium,17,17,21,21,[clt]
2523,rav,sfo,Mercedes-Benz Superdome,34,34,31,31,[nor]


#### Divisions

In [52]:
divisions_df = pd.read_csv('team_divisions.csv', index_col='idx')

In [53]:
before = games_lines_info_homeaway.shape[0]
for nteam in [0,1]:
    games_lines_info_homeaway = games_lines_info_homeaway.merge(
        divisions_df.rename(
            columns={'division': 'team{}_division'.format(nteam)}
        ),
        left_on=['season','team{}_fullname'.format(nteam)],
        right_on=['season','team']
    ).drop('team', axis=1)
    assert games_lines_info_homeaway.shape[0] == before

In [54]:
games_lines_info_homeaway[['team0_fullname','team0_division','team1_fullname','team1_division']].head()

,team0_fullname,team0_division,team1_fullname,team1_division
0,Indianapolis Colts,AFC South,New Orleans Saints,NFC South
1,Houston Texans,AFC South,New Orleans Saints,NFC South
2,Seattle Seahawks,NFC West,New Orleans Saints,NFC South
3,San Francisco 49ers,NFC West,New Orleans Saints,NFC South
4,Carolina Panthers,NFC South,New Orleans Saints,NFC South


#### Re-order columns

In [55]:
## skip re-naming
# homeaway_renames = dict(map(
#     lambda x: (x, x.replace('team0_','H_').replace('team1_','V_')),
#     games_lines_info_homeaway.columns
# ))

# games_lines_info_homeaway.rename(columns=homeaway_renames, inplace=True)

In [56]:
def reorder_cols(df, front_cols):
    '''takes a DF and list of columns 
    to move to the front (in order).
    returns DF'''
    return df[
        front_cols 
        + filter(
            lambda x: x not in front_cols, 
            df.columns
        )]
    
front_cols = ['game_id','season','week_id']
base_df = reorder_cols(games_lines_info_homeaway, front_cols)

In [57]:
base_df.head().T

,0,1,2,3,4
game_id,200709060clt,200711180htx,200710140sea,200710280sfo,200711250car
season,2007,2007,2007,2007,2007
week_id,0,10,5,7,11
team0_link,/teams/clt/2007.htm,/teams/htx/2007.htm,/teams/sea/2007.htm,/teams/sfo/2007.htm,/teams/car/2007.htm
team0_fullname,Indianapolis Colts,Houston Texans,Seattle Seahawks,San Francisco 49ers,Carolina Panthers
team0_name,Colts,Texans,Seahawks,49ers,Panthers
team0_city,Indianapolis,Houston,Seattle,San Francisco,Carolina
team0_score,41,23,17,10,6
team0_coach_raw,"<a href=""/coaches/DungTo0.htm"">Tony Dungy</a>","<a href=""/coaches/KubiGa0.htm"">Gary Kubiak</a>","<a href=""/coaches/HolmMi0.htm"">Mike Holmgren</a>","<a href=""/coaches/NolaMi0.htm"">Mike Nolan</a>","<a href=""/coaches/FoxxJo0.htm"">John Fox</a>"
team0_coach_name,Tony Dungy,Gary Kubiak,Mike Holmgren,Mike Nolan,John Fox


### Write Hive Table: __games_denorm__

In [58]:
pd.options.display.max_rows = 100

In [59]:
spark.createDataFrame(
        cast_dtypes(
            base_df
        )
    ).write.mode('overwrite').saveAsTable('games_denorm')

### Normalize and write to Hive tables (in __bold__)

In [60]:
## union team0 and team1 fields and get distinct values
def combine_team0_team1(full_df, all_fields, non_team_fields):
    '''given a DataFrame with 2 teams:
    * split into 2 DFs by team0 and team1
    * rename and align schemas
    * union
    * drop duplicates
    * return a DataFrame to be converted into Hive table'''

    for team_i in [0,1]:
        fields = [
            a for a in all_fields
            if 'team{}'.format(team_i) in a
        ]
        curr_table = full_df[non_team_fields + fields]
        ## team out team0 and team1 to align schemas
        field_map = dict(map(
                lambda x: (x, x.replace('team{}_'.format(team_i), '')
                               .replace('team{}'.format(team_i), 'team')
                          ), fields
            ))
        curr_table = curr_table.rename(columns=field_map)

        if team_i == 0:
            curr_df = curr_table
        else:
            curr_df = curr_df.append(
                curr_table[curr_df.columns]
            ).reset_index(drop=True)

    return curr_df.drop_duplicates()

#### __team_season__
* primary key is (season, link)
* joins with game

In [61]:
team_season_fields = [
    'season',
    'team0_link','team0_name','team0_fullname','team0_city','team0_division',
    'team1_link','team1_name','team1_fullname','team1_city','team1_division'
]

team_season_df = combine_team0_team1(
    base_df.copy(), team_season_fields, ['season']
)
## assert that every season has the same amount of teams
assert team_season_df['season'].value_counts().min() \
        == team_season_df['season'].value_counts().max()

## assert key (season, link) is unique
key = ['season','link']
assert team_season_df.groupby(key).size().max() == 1
team_season_df.head()

,season,link,name,fullname,city,division
0,2007,/teams/clt/2007.htm,Colts,Indianapolis Colts,Indianapolis,AFC South
1,2007,/teams/htx/2007.htm,Texans,Houston Texans,Houston,AFC South
2,2007,/teams/sea/2007.htm,Seahawks,Seattle Seahawks,Seattle,NFC West
3,2007,/teams/sfo/2007.htm,49ers,San Francisco 49ers,San Francisco,NFC West
4,2007,/teams/car/2007.htm,Panthers,Carolina Panthers,Carolina,NFC South


In [118]:
team_season_df['name'].value_counts().shape[0]
team_season_df['season'].value_counts()

32

2017    32
2016    32
2015    32
2014    32
2013    32
2012    32
2011    32
2010    32
2009    32
2008    32
2007    32
Name: season, dtype: int64

In [62]:
team_season_df.shape[0]
team_season_df.drop_duplicates(inplace=True)
team_season_df.shape[0]

352

352

In [63]:
spark.createDataFrame(
    cast_dtypes(team_season_df)
).write.mode('overwrite').saveAsTable('team_season')

In [64]:
base_df_remaining = set(base_df.columns) \
                    - set(team_season_df.columns.tolist()) \
                    | set(['season','team0_link','team1_link'])

#### __coach__
* primary key is coach_id
* joins with game

In [65]:
coach_fields = [
    'team0_coach_raw','team0_coach_name','team0_coach_id',
    'team1_coach_raw','team1_coach_name','team1_coach_id'
]

coach_df = combine_team0_team1(base_df.copy(), coach_fields, [])
## assert coach ID is unique
assert coach_df['coach_id'].value_counts().max() == 1
coach_df.head()

,coach_raw,coach_name,coach_id
0,"<a href=""/coaches/DungTo0.htm"">Tony Dungy</a>",Tony Dungy,/coaches/DungTo0.htm
1,"<a href=""/coaches/KubiGa0.htm"">Gary Kubiak</a>",Gary Kubiak,/coaches/KubiGa0.htm
2,"<a href=""/coaches/HolmMi0.htm"">Mike Holmgren</a>",Mike Holmgren,/coaches/HolmMi0.htm
3,"<a href=""/coaches/NolaMi0.htm"">Mike Nolan</a>",Mike Nolan,/coaches/NolaMi0.htm
4,"<a href=""/coaches/FoxxJo0.htm"">John Fox</a>",John Fox,/coaches/FoxxJo0.htm


In [66]:
coach_df.shape[0]
coach_df.drop_duplicates(inplace=True)
coach_df.shape[0]

92

92

In [67]:
spark.createDataFrame(
    cast_dtypes(coach_df)
).write.mode('overwrite').saveAsTable('coach')

In [68]:
base_df_remaining = set(base_df_remaining) \
                    - set(coach_fields) \
                    | set(['team0_coach_id','team1_coach_id'])

#### __team__
* primary key is (team_id, season)

In [69]:
key = ['team','season']
team_fields = ['team0_city','team0_division','team0_fullname',
               'team0_link','team0_name','team0',
               'team1_city','team1_division','team1_fullname',
               'team1_link','team1_name','team1']

team_df = combine_team0_team1(base_df.copy(), team_fields, ['season'])
# ## assert coach ID is unique
assert team_df.groupby(key).size().max() == 1
team_df.head()

,season,city,division,fullname,link,name,team
0,2007,Indianapolis,AFC South,Indianapolis Colts,/teams/clt/2007.htm,Colts,clt
1,2007,Houston,AFC South,Houston Texans,/teams/htx/2007.htm,Texans,htx
2,2007,Seattle,NFC West,Seattle Seahawks,/teams/sea/2007.htm,Seahawks,sea
3,2007,San Francisco,NFC West,San Francisco 49ers,/teams/sfo/2007.htm,49ers,sfo
4,2007,Carolina,NFC South,Carolina Panthers,/teams/car/2007.htm,Panthers,car


In [70]:
team_df.shape[0]
team_df.drop_duplicates(inplace=True)
team_df.shape[0]

352

352

In [71]:
spark.createDataFrame(
    cast_dtypes(team_df)
).write.mode('overwrite').saveAsTable('team')

In [72]:
base_df_remaining = set(base_df_remaining) \
                    - set(team_fields) \
                    | set(['team0','team1','season'])

#### __stadium__
* primary key is stadium name
* joins with game

In [73]:
## roof and surface are game metadata since it can change
## within a season
key = ['stadium']
stadium_fields = [
    'stadium_link','lat','long',
    'timezone','stadium_home_team'
]

stadium_df = base_df[key + stadium_fields].drop_duplicates()
## assert stadium name is unique
assert stadium_df.groupby(key).size().max() == 1
stadium_df.head()

,stadium,stadium_link,lat,long,timezone,stadium_home_team
0,RCA Dome,/stadiums/IND99.htm,39.76361,-86.16333,-5.0,['clt']
1,Reliant Stadium,/stadiums/HOU00.htm,29.68472,-95.41083,-6.0,['htx']
2,Qwest Field,/stadiums/SEA00.htm,47.59520,-122.33160,-8.0,['sea']
3,Monster Park,/stadiums/SFO00.htm,37.71361,-122.38611,-8.0,['sfo']
4,Bank of America Stadium,/stadiums/CAR00.htm,35.22583,-80.85278,-5.0,['car']


In [74]:
stadium_df.shape[0]
stadium_df.drop_duplicates(inplace=True)
stadium_df.shape[0]

61

61

In [75]:
spark.createDataFrame(
    cast_dtypes(stadium_df)
).write.mode('overwrite').saveAsTable('stadium')

In [76]:
base_df_remaining = set(base_df_remaining) \
                    - set(stadium_fields) \
                    | set(key)

#### change all team0 --> H, team1 --> V
* this is accurate
* can't move this code above as is, because previous parts depend on team0/team1 convention

In [77]:
## rename team0 and team1 to H and V
team_fields = filter(
    lambda x: x.startswith('team0_') or x.startswith('team1_'), 
    base_df.columns
)

team_field_rename = dict(
    map(
        lambda x: (x, x.replace('team0_','H_').replace('team1_','V_')), 
        team_fields
    )
)

team_field_rename['team0'] = 'H_team'
team_field_rename['team1'] = 'V_team'

base_df.rename(columns=team_field_rename, inplace=True)

In [78]:
## make same changes in list of remaining cols
base_df_remaining = map(
    lambda x: team_field_rename.get(x, x), base_df_remaining
)

In [79]:
base_df[sorted(list(base_df_remaining))].head().T

,0,1,2,3,4
2H_Fav,H,H,H,H,V
2H_OU,24.5,pk,23,17.5,20
2H_Spread,3.5,24.5,6.5,1,3
Close_Fav,H,H,H,V,V
Close_OU,52.5,49.5,43,39.5,42
Close_Spread,5.5,2,5.5,2,3
H_Final,41,23,17,10,6
H_ML,-240,-130,-250,115,155
H_Q1,7,7,0,0,3
H_Q2,3,10,10,0,3


#### __game_line__: TODO reorder cols

In [80]:
key = ['game_id']
line_fields = key + [
    '2H_Fav','2H_OU','2H_Spread','Close_Fav','Close_OU',
    'Close_Spread','H_ML','ML_Fav','Open_Fav','Open_OU',
    'Open_Spread','V_ML','pfr_line','pfr_ou']

lines = base_df[line_fields]
assert lines.groupby(key).size().max() == 1

In [81]:
lines.shape[0]
lines.drop_duplicates(inplace=True)
lines.shape[0]

2937

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


2937

In [82]:
spark.createDataFrame(
    cast_dtypes(lines)
).write.mode('overwrite').saveAsTable('game_line')

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [83]:
base_df_remaining = set(base_df_remaining) \
                    - set(line_fields) \
                    | set(key)
base_df[sorted(list(base_df_remaining))].head().T

,0,1,2,3,4
H_Final,41,23,17,10,6
H_Q1,7,7,0,0,3
H_Q2,3,10,10,0,3
H_Q3,14,0,0,3,0
H_Q4,17,6,7,7,0
H_coach_id,/coaches/DungTo0.htm,/coaches/KubiGa0.htm,/coaches/HolmMi0.htm,/coaches/NolaMi0.htm,/coaches/FoxxJo0.htm
H_score,41,23,17,10,6
H_team,clt,htx,sea,sfo,car
V_Final,10,10,28,31,31
V_Q1,0,3,7,10,0


#### __game_outcome__ (TODO: use V_score NOT V_Final... V_Final is wrong in 201610020pit game... messes up OT too... V_Q4 of that game is 14 not 7)

In [84]:
key = ['game_id']
outcome_fields = key + [
    'H_Final','H_Q1','H_Q2','H_Q3','H_Q4','H_score',
    'V_Final','V_Q1','V_Q2','V_Q3','V_Q4','V_score']

outcomes = base_df[outcome_fields]
assert outcomes.groupby(key).size().max() == 1

diffs are OT games

In [85]:
tmp1 = outcomes[outcomes[['H_Q1','H_Q2','H_Q3','H_Q4']].sum(axis=1) != outcomes['H_score']]
tmp1['diff'] = tmp1[['H_Q1','H_Q2','H_Q3','H_Q4']].sum(axis=1) - tmp1['H_score']
tmp1 = tmp1[['game_id','H_Q1','H_Q2','H_Q3','H_Q4','H_score','diff']]
tmp2 = outcomes[outcomes[['V_Q1','V_Q2','V_Q3','V_Q4']].sum(axis=1) != outcomes['V_score']]
tmp2['diff'] = tmp2[['V_Q1','V_Q2','V_Q3','V_Q4']].sum(axis=1) - tmp2['V_score']
tmp2 = tmp2[['game_id','V_Q1','V_Q2','V_Q3','V_Q4','H_score','diff']]

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [86]:
outcomes[outcomes['game_id'] == '201211180htx']

,game_id,H_Final,H_Q1,H_Q2,H_Q3,H_Q4,H_score,V_Final,V_Q1,V_Q2,V_Q3,V_Q4,V_score
429,201211180htx,43,7,10,3,14,43,37,7,10,10,7,37


In [87]:
tmp1.sort_values(by=['diff']).head()

,game_id,H_Q1,H_Q2,H_Q3,H_Q4,H_score,diff
429,201211180htx,7,10,3,14,43,-9
2345,201112040crd,3,0,3,7,19,-6
888,201111060crd,3,0,3,7,19,-6
605,201511290den,0,7,0,17,30,-6
2440,201701010pit,0,7,0,14,27,-6


In [88]:
tmp2.sort_values(by='diff').head()

,game_id,V_Q1,V_Q2,V_Q3,V_Q4,H_score,diff
2278,201610020pit,0,0,0,7,43,-7
75,200712090oti,0,0,3,14,17,-6
860,201012130htx,7,14,7,0,28,-6
1009,200710290den,7,6,0,0,13,-6
1143,200910110kan,0,3,7,10,20,-6


In [89]:
outcomes['H_OT'] = outcomes['H_score'] - outcomes[['H_Q1','H_Q2','H_Q3','H_Q4']].sum(axis=1)
outcomes['V_OT'] = outcomes['V_score'] - outcomes[['V_Q1','V_Q2','V_Q3','V_Q4']].sum(axis=1)

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [90]:
outcomes.shape[0]
outcomes.drop_duplicates(inplace=True)
outcomes.shape[0]

2937

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


2937

In [91]:
spark.createDataFrame(
    cast_dtypes(outcomes)
).write.mode('overwrite').saveAsTable('game_outcome')

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [92]:
base_df_remaining = set(base_df_remaining) \
                    - set(outcomes) \
                    | set(key)
base_df[sorted(list(base_df_remaining))].head().T

,0,1,2,3,4
H_coach_id,/coaches/DungTo0.htm,/coaches/KubiGa0.htm,/coaches/HolmMi0.htm,/coaches/NolaMi0.htm,/coaches/FoxxJo0.htm
H_team,clt,htx,sea,sfo,car
V_coach_id,/coaches/PaytSe0.htm,/coaches/PaytSe0.htm,/coaches/PaytSe0.htm,/coaches/PaytSe0.htm,/coaches/PaytSe0.htm
V_team,nor,nor,nor,nor,nor
attendance,57361,70780,68296,68244,72032
date,2007-09-06,2007-11-18,2007-10-14,2007-10-28,2007-11-25
day_of_week,3,6,6,6,6
duration,2:56,3:05,3:04,3:14,3:08
game_id,200709060clt,200711180htx,200710140sea,200710280sfo,200711250car
home_team,clt,htx,sea,sfo,car


#### __game_metadata__
* key is game_id

In [93]:
base_df['is_neutral'] = (base_df['home_team'] == 'neutral').astype(int)
base_df.drop('home_team', axis=1, inplace=True)

key = ['game_id']
metadata_fields = key + [
    'season','week_id','date','day_of_week','duration','prf_weather','roof',
    'surface','time','attendance','is_neutral']

metadata_df = base_df[metadata_fields]
assert metadata_df.groupby(key).size().max() == 1

In [94]:
metadata_df.shape[0]
metadata_df.drop_duplicates(inplace=True)
metadata_df.shape[0]

2937

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


2937

In [95]:
spark.createDataFrame(
    cast_dtypes(metadata_df)
).write.mode('overwrite').mode('overwrite').saveAsTable('game_metadata')

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [96]:
base_df_remaining = set(base_df_remaining) \
                    - set(metadata_fields) \
                    - set(['home_team']) \
                    | set(key)
base_df[sorted(list(base_df_remaining))].head().T

,0,1,2,3,4
H_coach_id,/coaches/DungTo0.htm,/coaches/KubiGa0.htm,/coaches/HolmMi0.htm,/coaches/NolaMi0.htm,/coaches/FoxxJo0.htm
H_team,clt,htx,sea,sfo,car
V_coach_id,/coaches/PaytSe0.htm,/coaches/PaytSe0.htm,/coaches/PaytSe0.htm,/coaches/PaytSe0.htm,/coaches/PaytSe0.htm
V_team,nor,nor,nor,nor,nor
game_id,200709060clt,200711180htx,200710140sea,200710280sfo,200711250car
stadium,RCA Dome,Reliant Stadium,Qwest Field,Monster Park,Bank of America Stadium


#### __game__
* primary key is game_id
* fact table

In [97]:
base_cols = sorted(list(base_df_remaining))
key = ['game_id']

game = base_df[key + list(set(base_cols) - set(key))]

In [98]:
game.shape[0]
game.drop_duplicates(inplace=True)
game.shape[0]

2937

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


2937

In [99]:
spark.createDataFrame(
    cast_dtypes(game)
).write.mode('overwrite').mode('overwrite').saveAsTable('game')

/Users/joshplotkin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [100]:
base_df_remaining = set(base_df_remaining) \
                    - set(game) \
                    - set(['home_team']) \
                    | set(key)
base_df[sorted(list(base_df_remaining))].head().T

,0,1,2,3,4
game_id,200709060clt,200711180htx,200710140sea,200710280sfo,200711250car


### __dvoa__
* primary key is (team, season)
* joins with team_season and game

In [133]:
dvoa_df = pd.read_csv('dvoa/dvoa_alltime.csv', index_col='Unnamed: 0')
## no cases in hive table
dvoa_df.columns = map(lambda x: x.replace(' ','_'), dvoa_df)
## headers are in data from appending dataframes. filter them out
dvoa_df.shape

(6112, 19)

#### temporarily remove 2018
* join key is (name, season)

In [134]:
dvoa_df = dvoa_df[dvoa_df['season'] != 2018]

In [135]:
dvoa_df['season'].value_counts()

2015    512
2011    512
2007    512
2014    512
2010    512
2017    512
2013    512
2009    512
2016    512
2012    512
2008    512
Name: season, dtype: int64

In [136]:
team_season = spark.table('team_season').cache()

In [137]:
team_season_years = set(
    team_season
        .groupby('season')
        .count()
        .toPandas()['season'].tolist()
)
dvoa_years = set(dvoa_df['season'].value_counts().index)

## assert years perfectly overlap
## 2018 missing from team_season_df
assert team_season_years == dvoa_years

In [138]:
team_season_teams = set(
    team_season
        .groupby('name')
        .count()
        .toPandas()['name'].tolist()
)
dvoa_teams = set(dvoa_df['name'].value_counts().index)

## assert team names perfectly overlap
assert team_season_teams == dvoa_teams

In [139]:
#### assert that it joins cleanly with game table
dvoa_size = dvoa_df.shape[0]
team_season_size = team_season.count()

key = ['name','season']
join_size = dvoa_df.merge(
    team_season.toPandas(), left_on=key, right_on=key
).shape[0]

## assert that dvoa_df hits all joins with team_season_df
## missing 2018 in team_season_df
assert dvoa_size == join_size

In [140]:
### convert % columns to floats
import numpy as np

pct_cols = [
    'TOTAL_DAVE','WEIGHTEDDVOA','TOTALDVOA','OFFENSEDVOA','DEFENSEDVOA',
    'S.T.DVOA','PAST_SCHED','FUTURE_SCHED','DAVE_OR_WTDDVOA','VAR'
]
for p in pct_cols:
    dvoa_df[p] = dvoa_df[p].str.replace('%','').astype(float).fillna(np.nan)

In [141]:
## assert 32 teams
tmp = dvoa_df['name'].value_counts()
assert tmp.shape[0] == 32
## assert every team plays the same amount of games
assert tmp.max() == tmp.min()

In [142]:
key = ['name','season']

team_names = spark.table('team').select(*(['team'] + key)).toPandas()
before = dvoa_df.shape[0]
dvoa_df = dvoa_df.merge(
        team_names, left_on=key, right_on=key
    ).drop(
        'name', axis=1
    )
assert dvoa_df.shape[0] == before

In [143]:
dvoa_df.shape[0]
dvoa_df.drop_duplicates(inplace=True)
dvoa_df.shape[0]

5632

5632

In [146]:
cols = ['team','season','week']
cols_rest = list(set(dvoa_df.columns) - set(cols))

dvoa_df = dvoa_df[cols + cols_rest]

dvoa_df.head().T

,0,1,2,3,4
team,pit,pit,pit,pit,pit
season,2007,2007,2007,2007,2007
week,1,2,3,4,5
WEIGHTEDDVOA,NaN,NaN,NaN,NaN,NaN
ESTIM.WINS,NaN,NaN,NaN,2.3,3.2
OFF.RANK,8,4,4,9,7
TOTALDVOA,92.4,82.3,79.9,34,39.4
OFFENSEDVOA,14.9,28.6,31.5,8.9,12.4
LASTWEEK,NaN,1,2,2,5
FUTURE_SCHED,NaN,NaN,NaN,-0.1,-0.5


In [147]:
spark.createDataFrame(
    cast_dtypes(dvoa_df)
).write.mode('overwrite').mode('overwrite').saveAsTable('dvoa')

### Examples of quick plotting via spark tables

In [ ]:
spark.table('line')\
    .groupby('Close_Spread')\
    .count()\
    .toPandas()\
    .set_index('Close_Spread')\
    .sort_index()\
    .plot(
        kind='bar', figsize=(16,8), color='#99d8c9', rot=0,
        title='Counts of Spread Magnitude'
    )

In [ ]:
spark.table('line')\
    .groupby('Close_Spread')\
    .pivot('Close_Fav')\
    .count()\
    .toPandas()\
    .set_index('Close_Spread')\
    .sort_index()\
    .plot(
        kind='bar', figsize=(16,8), color=['#af8dc3','#7fbf7b'], rot=0,
        title='Distributions of Home and Away Spreads'
    )